<div dir="rtl" style="text-align: justify"><font face="B Nazanin" size=5>
توضیحات :
در این تمرین هدف پیاده سازی deformable conv و مقایسه آن با conv معمولی است.

<br>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CocoDetection
from torch.optim import SGD
import time
import os
import numpy as np

<div dir="rtl" style="text-align: justify"><font face="B Nazanin" size=4>
توضیحات :
در این قسمت با توجه به خواسته مسئله ( تغییر یافته MS coco به Mnist ) ابتدا دیتاست Mnist فراخوانی می شود.
سپس دیتاست و لودر آن ساخته می شود.

<br>

In [ ]:
import torchvision
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

# Define transforms for MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Define data paths for MNIST (no annotation files needed for MNIST)
train_data_path = 'data/MNIST/train'
val_data_path = 'data/MNIST/test'

# Create custom datasets for MNIST
train_dataset = MNIST(root=train_data_path, train=True, download=True, transform=transform)
val_dataset = MNIST(root=val_data_path, train=False, download=True, transform=transform)

# Create data loaders for MNIST
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


100%|██████████| 9912422/9912422 [00:00<00:00, 111018067.65it/s]


Extracting data/MNIST/train/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/train/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 45522620.75it/s]

Extracting data/MNIST/train/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/train/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 25049318.51it/s]


...

100%|██████████| 1648877/1648877 [00:00<00:00, 34332605.55it/s]


Extracting data/MNIST/test/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/test/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6808623.58it/s]

Extracting data/MNIST/test/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/test/MNIST/raw




/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


<div dir="rtl" style="text-align: justify"><font face="B Nazanin" size=4>
توضیحات :
در این قسمت با توجه به هدف مسئله deformable conv پیاده سازی می شود.

در اینجا یک لایهٔ کانولوشن (offset_conv) برای محاسبهٔ آفست‌ها ایجاد می‌شود. این لایه دو برابر اندازه کرنل kernel_size دارد تا بتواند دو مقدار برای هر نقطه در ورودی تولید کند.
<br>

<div dir="rtl" style="text-align: justify"><font face="B Nazanin" size=4>
لایهٔ دوم (modulator_conv) برای محاسبهٔ مدولاتورها ایجاد می‌شود. مدولاتورها از تابع فعال‌سازی sigmoid عبور داده می‌شوند.
<br>
<div dir="rtl" style="text-align: justify"><font face="B Nazanin" size=4>
لایهٔ سوم (regular_conv) به عنوان لایهٔ کانولوشن معمولی استفاده می‌شود.

<br>

In [ ]:
#%% based on https://github.com/developer0hye/PyTorch-Deformable-Convolution-v2/blob/main/dcn.py
import torch
import torchvision.ops
from torch import nn


class DeformableConv2d(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride=1,
                 padding=1,
                 dilation=1,
                 bias=False):
        super(DeformableConv2d, self).__init__()

        assert type(kernel_size) == tuple or type(kernel_size) == int

        kernel_size = kernel_size if type(kernel_size) == tuple else (kernel_size, kernel_size)
        self.stride = stride if type(stride) == tuple else (stride, stride)
        self.padding = padding
        self.dilation = dilation

        self.offset_conv = nn.Conv2d(in_channels,
                                     2 * kernel_size[0] * kernel_size[1],
                                     kernel_size=kernel_size,
                                     stride=stride,
                                     padding=self.padding,
                                     dilation=self.dilation,
                                     bias=True)

        nn.init.constant_(self.offset_conv.weight, 0.)
        nn.init.constant_(self.offset_conv.bias, 0.)

        self.modulator_conv = nn.Conv2d(in_channels,
                                        1 * kernel_size[0] * kernel_size[1],
                                        kernel_size=kernel_size,
                                        stride=stride,
                                        padding=self.padding,
                                        dilation=self.dilation,
                                        bias=True)

        nn.init.constant_(self.modulator_conv.weight, 0.)
        nn.init.constant_(self.modulator_conv.bias, 0.)

        self.regular_conv = nn.Conv2d(in_channels=in_channels,
                                      out_channels=out_channels,
                                      kernel_size=kernel_size,
                                      stride=stride,
                                      padding=self.padding,
                                      dilation=self.dilation,
                                      bias=bias)

    def forward(self, x):
        # h, w = x.shape[2:]
        # max_offset = max(h, w)/4.

        offset = self.offset_conv(x)  # .clamp(-max_offset, max_offset)
        modulator = 2. * torch.sigmoid(self.modulator_conv(x))
        # op = (n - (k * d - 1) + 2p / s)
        x = torchvision.ops.deform_conv2d(input=x,
                                          offset=offset,
                                          weight=self.regular_conv.weight,
                                          bias=self.regular_conv.bias,
                                          padding=self.padding,
                                          mask=modulator,
                                          stride=self.stride,
                                          dilation=self.dilation)
        return x

<div dir="rtl" style="text-align: justify"><font face="B Nazanin" size=4>
در این قسمت یک مدل ساده براساس def conv و conv معمولی نوشته می شود.
هدف مقایسه بین کارکرد دو نوع conv است.

<br>

In [ ]:
# Deformable Convolution Model
class DeformableConvModel(nn.Module):
    def __init__(self, kernel_size):
        super(DeformableConvModel, self).__init__()

        self.deformable_conv = DeformableConv2d(in_channels=1, out_channels=64, kernel_size=kernel_size, stride=1, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=kernel_size, stride=2)

        self.fc = nn.Linear(outputsize(outputsize(28 , kernel_size , 1 , 1), kernel_size , 2, 0)*outputsize(outputsize(28 , kernel_size , 1 , 1), kernel_size , 2, 0) * 64, 10)  # Output size is 10 for MNIST classes

    def forward(self, x):
        x = self.deformable_conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Regular Convolution Model
class RegularConvModel(nn.Module):
    def __init__(self, kernel_size):
        super(RegularConvModel, self).__init__()

        self.conv = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=kernel_size, stride=1, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=kernel_size, stride=2)

        self.fc = nn.Linear(outputsize(outputsize(28 , kernel_size , 1 , 1), kernel_size , 2, 0)*outputsize(outputsize(28 , kernel_size , 1 , 1), kernel_size , 2, 0) * 64, 10)  # Output size is 10 for MNIST classes

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
def outputsize(I , k , s , p) :
  o = int((np.floor((I-k+2*p)/s))+1)
  return o

<div dir="rtl" style="text-align: justify"><font face="B Nazanin" size=4>
در این قسمت loss مورد نیاز تعریف می شود.
<br>

In [ ]:
criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



<div dir="rtl" style="text-align: justify"><font face="B Nazanin" size=4>
در این قسمت دو تابع جهت train , eval مدل نوشته می شود.
روند نوشتن توابع کاملا روش روتین ترینینگ است به طوری که مقدار مدل و نتیجه واقعی مقایسه شده و وزن ها آپدیت می شوند.
<br>

In [ ]:
from tqdm import tqdm

# Training loop
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()

    # Wrap your train_loader with tqdm for the progress bar
    for inputs, targets in tqdm(train_loader, desc="Training", leave=False):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    duration = time.time() - start_time
    accuracy = correct / total
    average_loss = total_loss / len(train_loader)

    return accuracy, average_loss, duration

# Validation loop
def evaluate_model(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()

    # Wrap your val_loader with tqdm for the progress bar
    for inputs, targets in tqdm(val_loader, desc="Validation", leave=False):
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    duration = time.time() - start_time
    accuracy = correct / total
    average_loss = total_loss / len(val_loader)

    return accuracy, average_loss, duration


<div dir="rtl" style="text-align: justify"><font face="B Nazanin" size=4>
در نهایت 2 مدل به ازای kernel size های مختلف جهت مقایسه اثر grid sampling size مقایسه می شوند.
به طوری که به ازای 5 ایپاک نتایج گزارش می شود.
<br>

In [ ]:

for kernel_size in range(2 , 5):
  print('kernel_size = ' + str(kernel_size)+'\n')
  # Training and Evaluation
  num_epochs = 5
  # Instantiate models
  deformable_model = DeformableConvModel(kernel_size = kernel_size).to(device)
  regular_model = RegularConvModel(kernel_size = kernel_size).to(device)

  # Loss function and optimizer
  deformable_optimizer = SGD(deformable_model.parameters(), lr=0.001, momentum=0.9)
  regular_optimizer = SGD(regular_model.parameters(), lr=0.001, momentum=0.9)
  for epoch in range(num_epochs):
      print('epoch = '+str(epoch+1)+ '\n')
      # Train Deformable Conv Model
      deformable_train_accuracy, deformable_train_loss, deformable_train_duration = train_model(deformable_model, train_loader, criterion, deformable_optimizer, device)

      # Evaluate Deformable Conv Model on Validation Set
      deformable_val_accuracy, deformable_val_loss, deformable_val_duration = evaluate_model(deformable_model, val_loader, criterion, device)

      # Train Regular Conv Model
      regular_train_accuracy, regular_train_loss, regular_train_duration = train_model(regular_model, train_loader, criterion, regular_optimizer, device)

      # Evaluate Regular Conv Model on Validation Set
      regular_val_accuracy, regular_val_loss, regular_val_duration = evaluate_model(regular_model, val_loader, criterion, device)

      # Print or log the metrics
      print(f'Epoch [{epoch + 1}/{num_epochs}]')
      print('Deformable Conv Model:')
      print(f'Training Accuracy: {deformable_train_accuracy:.4f}, Training Loss: {deformable_train_loss:.4f}, Training Duration: {deformable_train_duration:.2f}s')
      print(f'Validation Accuracy: {deformable_val_accuracy:.4f}, Validation Loss: {deformable_val_loss:.4f}, Validation Duration: {deformable_val_duration:.2f}s')

      print('Regular Conv Model:')
      print(f'Training Accuracy: {regular_train_accuracy:.4f}, Training Loss: {regular_train_loss:.4f}, Training Duration: {regular_train_duration:.2f}s')
      print(f'Validation Accuracy: {regular_val_accuracy:.4f}, Validation Loss: {regular_val_loss:.4f}, Validation Duration: {regular_val_duration:.2f}s')
      print('\n')

  # After training, you can use the trained models for further analysis or testing.



kernel_size = 2

epoch = 1



Epoch [1/5]
Deformable Conv Model:
Training Accuracy: 0.8855, Training Loss: 0.4198, Training Duration: 14.93s
Validation Accuracy: 0.9172, Validation Loss: 0.2829, Validation Duration: 1.82s
Regular Conv Model:
Training Accuracy: 0.8781, Training Loss: 0.4317, Training Duration: 12.39s
Validation Accuracy: 0.9141, Validation Loss: 0.2920, Validation Duration: 1.73s


epoch = 2



Epoch [2/5]
Deformable Conv Model:
Training Accuracy: 0.9214, Training Loss: 0.2709, Training Duration: 15.05s
Validation Accuracy: 0.9326, Validation Loss: 0.2398, Validation Duration: 2.65s
Regular Conv Model:
Training Accuracy: 0.9162, Training Loss: 0.2871, Training Duration: 11.20s
Validation Accuracy: 0.9291, Validation Loss: 0.2463, Validation Duration: 2.30s


epoch = 3



Epoch [3/5]
Deformable Conv Model:
Training Accuracy: 0.9351, Training Loss: 0.2265, Training Duration: 13.66s
Validation Accuracy: 0.9441, Validation Loss: 0.1952, Validation Duration: 1.81s
Regular Conv Model:
Training Accuracy: 0.9254, Training Loss: 0.2564, Training Duration: 11.94s
Validation Accuracy: 0.9338, Validation Loss: 0.2335, Validation Duration: 1.71s


epoch = 4



Epoch [4/5]
Deformable Conv Model:
Training Accuracy: 0.9477, Training Loss: 0.1862, Training Duration: 13.67s
Validation Accuracy: 0.9553, Validation Loss: 0.1600, Validation Duration: 1.84s
Regular Conv Model:
Training Accuracy: 0.9329, Training Loss: 0.2331, Training Duration: 14.23s
Validation Accuracy: 0.9370, Validation Loss: 0.2171, Validation Duration: 1.68s


epoch = 5



...

Epoch [1/5]
Deformable Conv Model:
Training Accuracy: 0.9141, Training Loss: 0.3121, Training Duration: 14.07s
Validation Accuracy: 0.9625, Validation Loss: 0.1406, Validation Duration: 2.67s
Regular Conv Model:
Training Accuracy: 0.9118, Training Loss: 0.3233, Training Duration: 12.65s
Validation Accuracy: 0.9586, Validation Loss: 0.1553, Validation Duration: 2.18s


epoch = 2



Epoch [2/5]
Deformable Conv Model:
Training Accuracy: 0.9649, Training Loss: 0.1253, Training Duration: 14.26s
Validation Accuracy: 0.9685, Validation Loss: 0.1008, Validation Duration: 2.20s
Regular Conv Model:
Training Accuracy: 0.9622, Training Loss: 0.1373, Training Duration: 12.12s
Validation Accuracy: 0.9723, Validation Loss: 0.1018, Validation Duration: 1.91s


epoch = 3



Epoch [3/5]
Deformable Conv Model:
Training Accuracy: 0.9741, Training Loss: 0.0917, Training Duration: 13.94s
Validation Accuracy: 0.9789, Validation Loss: 0.0710, Validation Duration: 1.83s
Regular Conv Model:
Training Accuracy: 0.9717, Training Loss: 0.1013, Training Duration: 12.02s
Validation Accuracy: 0.9769, Validation Loss: 0.0823, Validation Duration: 1.75s


epoch = 4



Epoch [4/5]
Deformable Conv Model:
Training Accuracy: 0.9778, Training Loss: 0.0761, Training Duration: 13.95s
Validation Accuracy: 0.9788, Validation Loss: 0.0749, Validation Duration: 1.78s
Regular Conv Model:
Training Accuracy: 0.9768, Training Loss: 0.0837, Training Duration: 12.08s
Validation Accuracy: 0.9784, Validation Loss: 0.0708, Validation Duration: 1.70s


epoch = 5



Epoch [5/5]
Deformable Conv Model:
Training Accuracy: 0.9809, Training Loss: 0.0665, Training Duration: 13.91s
Validation Accuracy: 0.9822, Validation Loss: 0.0584, Validation Duration: 1.80s
Regular Conv Model:
Training Accuracy: 0.9795, Training Loss: 0.0728, Training Duration: 12.12s
Validation Accuracy: 0.9810, Validation Loss: 0.0619, Validation Duration: 1.71s




<div dir="rtl" style="text-align: justify"><font face="B Nazanin" size=4>
همان طور که از نتایج مشاهده می شود. deformable conv در ایپاک کم تر به نتیجه بهتر از conv معمولی می رسد گرچه زمان training هر مرحله در def conv بیشتر  طول می کشد.
<br>